In [8]:
import nilearn
from nilearn import image, datasets, plotting, masking
from pathlib import Path
import pandas as pd
import numpy as np
#import meld
from matplotlib import pyplot as plt
#from numba import jit
from pathlib import Path
import subprocess
import os
import re
%matplotlib inline

In [3]:
project_dir = Path('/data/nielsond/meld/')
%pwd
%cd {project_dir}
%pwd

/spin1/users/nielsond/meld


'/spin1/users/nielsond/meld'

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', 500)
def run(command, env={}, shell=False):
    merged_env = os.environ
    merged_env.update(env)
    process = subprocess.Popen(command, stdout=subprocess.PIPE,
                               stderr=subprocess.STDOUT, shell=shell,
                               env=merged_env)
    while True:
        line = process.stdout.readline()
        line = str(line, 'utf-8')[:-1]
        print(line)
        if line == '' and process.poll() is not None:
            break
    if process.returncode != 0:
        raise Exception("Non zero return code: %d"%process.returncode)

In [79]:
bids_dir = Path('data/self_con')
proc_dir = Path('data/self_con/derivatives/afni_proc2')
swarm_path = Path('proc_swarm_meld_self_con.cmd')
log_dir = Path('swarm_log')

In [158]:
# save out single_trial_betas
subj_img_nums = []
beta_inds = {'att_neg':list(range(20)),
             'att_neut': list(range(20)),
             'suppr_neg': list(range(20))}
cmds = []
stb_df = []
for sub_dir in sorted(bids_dir.glob('sub-*')):
    subj = re.findall('.*/sub-([0-9]*)', sub_dir.as_posix())[0]
    stim_dict = {}
    rn = 0
    stim_df = []
    image_nums = {}
    trial_nums = {}
    run_nums = {}
    subn = sub_dir.parts[-1][4:]
    stb_dir = proc_dir/sub_dir.parts[-1]/'emotionalregulation/stb/'
    if not stb_dir.exists():
        stb_dir.mkdir()
    for run_file in sorted((bids_dir/('sub-%s'%subj)).glob('func/*emotionalregulation*events.tsv')):
        stim_df = pd.read_csv(run_file, sep = '\t')
        run_n = int(re.findall('.*_run-([0-9]*)', run_file.as_posix())[0])
        for tt in np.unique(stim_df.trial_type_orig):
            if tt in ['att_neg','att_neut','suppr_neg']:
                try:
                    image_nums[tt] = np.hstack([image_nums[tt],stim_df.query('trial_type_orig == @tt').image_num.astype(int).values])
                    trial_nums[tt] = np.hstack([trial_nums[tt],stim_df.query('trial_type_orig == @tt').trial_num.astype(int).values])
                    run_nums[tt].extend([run_n] * len(stim_df.query('trial_type_orig == @tt').trial_num))
                except KeyError:
                    image_nums[tt] = stim_df.query('trial_type_orig == @tt').image_num.astype(int).values
                    trial_nums[tt] = stim_df.query('trial_type_orig == @tt').trial_num.astype(int).values
                    run_nums[tt] = [run_n] * len(stim_df.query('trial_type_orig == @tt').trial_num)

        subj_img_nums.append(image_nums)
    for tt,inds in beta_inds.items():
        for run_num, trial_num, img_num, ii in zip(run_nums[tt], trial_nums[tt], image_nums[tt], inds):
            reml_path = proc_dir/sub_dir.parts[-1]/('emotionalregulation/stats.{subn}_REML+tlrc.HEAD[{ii}]'.format(subn = subn, ii = "'%s#%d_Coef'"%(tt,ii)))
            out_path = stb_dir/'REML_{stim_type}_img-{image_num}_run-{rn}_trial-{n}.nii.gz'.format(stim_type = tt, n = "%03d"%trial_num,rn = run_num, image_num = "%03d"%img_num) 
            cmd = 'module load afni; 3dAFNItoNIFTI -overwrite -prefix {out} {in1}'.format(out = out_path, in1 = reml_path)
            stb_dat = {}
            stb_dat['subject'] = subn
            stb_dat['trial_type'] = tt
            stb_dat['image_num'] = img_num
            stb_dat['trial_num'] = trial_num
            stb_dat['run_num'] = run_num
            stb_dat['path'] = out_path.absolute()
            cmds.append(cmd)
            #run(cmd, shell = True)
            stb_df.append(stb_dat)

In [140]:
swarm_path.write_text('\n'.join(cmds[:2]))
                      
print(swarm_path.read_text())

3dAFNItoNIFTI -overwrite -prefix data/self_con/derivatives/afni_proc2/sub-01/emotionalregulation/stb/REML_att_neg_img-071_run-1_trial-002.nii.gz data/self_con/derivatives/afni_proc2/sub-01/emotionalregulation/stats.01_REML+tlrc.HEAD['att_neg\#0_Coef']
3dAFNItoNIFTI -overwrite -prefix data/self_con/derivatives/afni_proc2/sub-01/emotionalregulation/stb/REML_att_neg_img-077_run-1_trial-003.nii.gz data/self_con/derivatives/afni_proc2/sub-01/emotionalregulation/stats.01_REML+tlrc.HEAD['att_neg\#1_Coef']


In [146]:
if not log_dir.exists():
    log_dir.mkdir()

!swarm -f {swarm_path} --module afni -g 2 -t 2 -b2 --logdir {log_dir} --time 00:05:00 --partition=nimh,norm,quick

56671260


In [161]:
stb_df = pd.DataFrame(stb_df)

In [162]:
stb_df['file_exists'] = [pp.exists() for pp in stb_df.path.values]

In [163]:
stb_df.query('file_exists == False')

,image_num,path,run_num,subject,trial_num,trial_type,file_exists


In [4]:
base_dir = Path('../data/voice/')
proc_dir = base_dir/'derivatives/afni_proc2'

In [9]:
!which 3dAFNItoNIFTI

/usr/local/apps/afni/current/linux_openmp_64/3dAFNItoNIFTI


In [20]:
#convert REMLs into nifits
bad_list = ['sub-007', 'sub-023']
for sub_dir in sorted(proc_dir.glob('sub-*')):
    sub = sub_dir.parts[-1]
    if sub not in bad_list:
        subn = sub_dir.parts[-1][4:]
        stb_dir = sub_dir/'passivelistening/stb/'
        try:
            if not stb_dir.exists():
                stb_dir.mkdir()
                stim_type = 'non_vocal'
                for ii in range(1,41,2):
                    reml_path = sub_dir/('passivelistening/stats.{subn}_REML+tlrc.HEAD[{ii}]'.format(subn = subn, ii = ii))
                    out_path = stb_dir/'REML_{stim_type}_{n}.nii.gz'.format(stim_type = stim_type, n = int((ii-1)/2 + 1))
                    cmd = '3dAFNItoNIFTI -overwrite -prefix {out} {in1}'.format(out = out_path, in1 = reml_path)
                    run(cmd, shell = True)
                stim_type = 'vocal'
                for ii in range(42,82,2):
                    reml_path = sub_dir/('passivelistening/stats.{subn}_REML+tlrc.HEAD[{ii}]'.format(subn = subn, ii = ii))
                    out_path = stb_dir/'REML_{stim_type}_{n}.nii.gz'.format(stim_type = stim_type, n = int((ii-42)/2 + 1))
                    cmd = '3dAFNItoNIFTI -overwrite -prefix {out} {in1}'.format(out = out_path, in1 = reml_path)
                    run(cmd, shell = True)
        except:
            bad_list.append(sub)

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec


++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (De


++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (De


++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (De


++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (De

...


++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (De


++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (De


++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (De


++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (De

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec

In [21]:
bad_list

['sub-007', 'sub-023', 'sub-072', 'sub-077', 'sub-137', 'sub-165']

In [25]:
dat_list = []
for sub_dir in sorted(proc_dir.glob('sub-*')):
    sub = sub_dir.parts[-1]
    if sub not in bad_list:
        subn = sub_dir.parts[-1][4:]
        afni_mask_path = sub_dir/('passivelistening/full_mask.{subn}+tlrc.HEAD'.format(subn = subn))
        nii_mask_path = sub_dir/('passivelistening/full_mask.{subn}+tlrc.nii.gz'.format(subn = subn))
        cmd = '3dAFNItoNIFTI -overwrite -prefix {out} {in1}'.format(out = nii_mask_path, in1 = afni_mask_path)
        run(cmd,shell = True)

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec


++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (Dec 11 2017) [64-bit]

++ 3dAFNItoNIFTI: AFNI version=AFNI_17.3.07 (De

In [ ]:
../data/voice/derivatives/afnie_

In [34]:
cmd = '3dmask_tool -overwrite -prefix ../data/voice/derivatives/afni_proc2/group_full_mask.nii.gz  -frac 0 -dilate_result 2 -2 -inputs ../data/voice/derivatives/afni_proc2/sub-*/passivelistening/full_mask.*.HEAD'
run(cmd, shell = True)

++ processing 216 input datasets...
++ padding all datasets by 0 (for dilations)
++ frac 0 over 216 volumes gives min count 0
++ voxel limits: 0 clipped, 98547 survived, 212749 were zero
++ writing result group_full_mask.nii.gz...
++ Output dataset ../data/voice/derivatives/afni_proc2/group_full_mask.nii.gz



In [25]:
run

PosixPath('/meld_root/work/data/voice/derivatives/afni_proc2/sub-001/passivelistening/stats.001_REML+tlrc.HEAD[39]')